In [1]:
#Dependencies
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
#SQL Alchemy path 
database_path = "Resources/fl_voter_data.sqlite"

In [3]:
#SQL Alchemy Engine
engine = create_engine(f'sqlite:///{database_path}', echo=False)

In [4]:
flvd_df = pd.read_csv("Resources/fl_voter_data_2022.csv")
#Check initial dimensionality
flvd_df.shape

(834, 11)

In [5]:
#Drop extranenous header rows created by an image at the top of the original excel.
flvd_df=flvd_df.drop([0,1,2,3,4,5,6])
#Recheck dimensionality
flvd_df.shape

(827, 11)

In [6]:
#Get column names for renaming:
for col in flvd_df.columns:
    print(col)

FLORIDA DEPARTMENT OF STATE
Unnamed: 1
Unnamed: 2
Unnamed: 3
Unnamed: 4
Unnamed: 5
Unnamed: 6
Unnamed: 7
Unnamed: 8
Unnamed: 9
Unnamed: 10


In [7]:
#Rename columns
flvd_df=flvd_df.rename(columns={"FLORIDA DEPARTMENT OF STATE": "Party",
                       "Unnamed: 1": "County_Name",
                       "Unnamed: 2": "Native",
                       "Unnamed: 3": "Asian",
                       "Unnamed: 4": "Black",         
                       "Unnamed: 5": "Hispanic",
                       "Unnamed: 6": "White",
                       "Unnamed: 7": "Other",
                       "Unnamed: 8": "Multi",
                       "Unnamed: 9": "Unknown",
                       "Unnamed: 10": "Total_Voters"
                       })
#Check column rename
for col in flvd_df.columns:
    print(col)

Party
County_Name
Native
Asian
Black
Hispanic
White
Other
Multi
Unknown
Total_Voters


# Eliminating null values

In [8]:
#Drop empty rows and check dimensionality
flvd_df.dropna(inplace=True)
flvd_df.shape

(816, 11)

In [9]:
#Confirm there are no null values
flvd_df.isnull().sum()

Party           0
County_Name     0
Native          0
Asian           0
Black           0
Hispanic        0
White           0
Other           0
Multi           0
Unknown         0
Total_Voters    0
dtype: int64

# Convert Columns to Usable Datatypes

In [10]:
#Check column datatypes
flvd_df.dtypes

Party           object
County_Name     object
Native          object
Asian           object
Black           object
Hispanic        object
White           object
Other           object
Multi           object
Unknown         object
Total_Voters    object
dtype: object

In [11]:
#The Race/Ethniciy columns contain numeric data that was created as text that includes commas.
#Remove commas
flvd_df[['Native','Asian','Black','Hispanic','White','Other','Multi','Unknown','Total_Voters']] = flvd_df[['Native','Asian','Black','Hispanic','White','Other','Multi','Unknown','Total_Voters']].replace(',','', regex=True)
#Change object columns to numeric
flvd_df[['Native','Asian','Black','Hispanic','White','Other','Multi','Unknown','Total_Voters']] = flvd_df[['Native','Asian','Black','Hispanic','White','Other','Multi','Unknown','Total_Voters']].apply(pd.to_numeric)

In [12]:
#Confirm changes to column datatypes
flvd_df.dtypes

Party           object
County_Name     object
Native           int64
Asian            int64
Black            int64
Hispanic         int64
White            int64
Other            int64
Multi            int64
Unknown          int64
Total_Voters     int64
dtype: object

# Inspect and Clean Object Datatype Columns

### Clean County Names

In [13]:
#Inspect County Names
flvd_df['County_Name'].unique()

array(['Alachua                  ', 'Baker                    ',
       'Bay                      ', 'Bradford                 ',
       'Brevard                  ', 'Broward                  ',
       'Calhoun                  ', 'Charlotte                ',
       'Citrus                   ', 'Clay                     ',
       'Collier                  ', 'Columbia                 ',
       'DeSoto                   ', 'Dixie                    ',
       'Duval                    ', 'Escambia                 ',
       'Flagler                  ', 'Franklin                 ',
       'Gadsden                  ', 'Gilchrist                ',
       'Glades                   ', 'Gulf                     ',
       'Hamilton                 ', 'Hardee                   ',
       'Hendry                   ', 'Hernando                 ',
       'Highlands                ', 'Hillsborough             ',
       'Holmes                   ', 'Indian River             ',
       'Jackson          

In [14]:
#Drop rows in 'County Name' where the name equals 'Total'
#This is due to the orignial file contain Total rows for Partys.
#Previous row check was 816
totindx=flvd_df[flvd_df['County_Name']=='Total'].index
flvd_df.drop(totindx, inplace=True)

In [15]:
#Count of unique party names
flvd_df['Party'].nunique()

12

In [16]:
#816 - 12 is 804 which should match in a new dimensionality check
flvd_df.shape

(804, 11)

In [17]:
# Per https://www2.census.gov/geo/pdfs/reference/GARM/Ch4GARM.pdf , the number of Florida counties is 67
# This matches the return from nunique().
flvd_df['County_Name'].nunique()

67

### Clean Party Names

In [18]:
#Examine strings in 'Party' column
flvd_df['Party'].unique()

array(['Republican Party of Florida                                        ',
       'Florida Democratic Party                                            ',
       'Coalition With A Purpose Party', 'Constitution Party of Florida',
       'Ecology Party of Florida',
       'Green Party of Florida                 ',
       'Independent Party of Florida', 'Libertarian Party of Florida',
       'Party for Socialism and Liberation - Florida',
       "People's Party of Florida", 'Reform Party of Florida',
       'No Party Affiliation          '], dtype=object)

In [19]:
# Function to standarize 'Party' to "REP", "DEM", and "UNA"
# 'UNA' will be used to sum the counts of all other political parties, including "No affiliation",
# that are not Republican or Democrat.
def rename_party(party):
    #if party = Repub... then REP
    if party =='Republican Party of Florida                                        ':
        return "REP"
    #elif party = Democ... then DEM
    elif "Democrat" in party: 
        return "DEM"
    #else then UNA
    else:
        return "UNA"
#                                          apply(lambda x:rename_party(x))
#.copy() is used to avoid a SettingWithCopyWarning
flvd_df['Party'] = flvd_df['Party'].apply(lambda party:rename_party(party))
#lambda "says" we are taking the value from a function and then .apply to the column
#It's a way for a function to exist temporarily.

In [20]:
#Confirm changes to data in 'Party' column
flvd_df['Party'].unique()

array(['REP', 'DEM', 'UNA'], dtype=object)

In [21]:
#Add a column for state and populate the column with 'FL' in each row.
flvd_df['State']= 'FL'

In [22]:
#Examine dataframe
flvd_df

,Party,County_Name,Native,Asian,Black,Hispanic,White,Other,Multi,Unknown,Total_Voters,State
7,REP,Alachua,168,1071,968,2720,42176,1039,180,326,48648,FL
8,REP,Baker,57,56,72,130,10636,84,19,82,11136,FL
9,REP,Bay,311,945,581,1294,62423,570,123,929,67176,FL
10,REP,Bradford,35,39,95,113,9652,70,19,61,10084,FL
11,REP,Brevard,701,3070,1983,8660,176889,2262,506,1935,196006,FL
...,...,...,...,...,...,...,...,...,...,...,...,...
828,UNA,Union,8,5,53,34,631,9,6,28,774,FL
829,UNA,Volusia,380,2195,7469,16989,84288,2850,664,4100,118935,FL
830,UNA,Wakulla,21,37,275,103,3840,53,19,114,4462,FL
831,UNA,Walton,42,206,368,517,10567,192,42,367,12301,FL


In [23]:
#Reset Dataframe index
flvd_df.reset_index(drop=True, inplace=True)


In [24]:
flvd_df.index = flvd_df.index + 1

In [25]:
#Confirm index starts at 1
flvd_df.head(5)

,Party,County_Name,Native,Asian,Black,Hispanic,White,Other,Multi,Unknown,Total_Voters,State
1,REP,Alachua,168,1071,968,2720,42176,1039,180,326,48648,FL
2,REP,Baker,57,56,72,130,10636,84,19,82,11136,FL
3,REP,Bay,311,945,581,1294,62423,570,123,929,67176,FL
4,REP,Bradford,35,39,95,113,9652,70,19,61,10084,FL
5,REP,Brevard,701,3070,1983,8660,176889,2262,506,1935,196006,FL


In [26]:
#Confirm dataframe is working with some light analysis: County_Name
votes_by_county = flvd_df.groupby('County_Name')['Total_Voters'].sum().sort_values(ascending=False)
votes_by_county

County_Name
Miami-Dade                   1529401
Broward                      1252774
Palm Beach                   1003667
Hillsborough                  924891
Orange                        870296
                              ...   
Hamilton                        7846
Union                           7643
Glades                          6959
Lafayette                       4544
Liberty                         4452
Name: Total_Voters, Length: 67, dtype: int64

In [27]:
#Confirm dataframe is working with some light analysis: Party
votes_by_county = flvd_df.groupby('Party')['Total_Voters'].sum().sort_values(ascending=False)
votes_by_county

Party
REP    5277394
DEM    4971444
UNA    4255140
Name: Total_Voters, dtype: int64

In [28]:
#Convert dataframe to SQL table
#database_path = "Resources/fl_voter_data.sqlite"
flvd_df.to_sql('Resources/FL_Voters', con=engine, if_exists='replace')

804

In [31]:
#Examine SQL Table
SQLtable = engine.execute("SELECT * FROM FL_Voters")
for rows in SQLtable:
    print(rows)

OperationalError: (sqlite3.OperationalError) no such table: FL_Voters
[SQL: SELECT * FROM FL_Voters]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
flvd_df2.to_csv("db_output/ETL_FL_voterdata_clean.csv")